<a href="https://colab.research.google.com/github/rechavar/pyCovid_TechLabsProj/blob/Luca/TechLabs__finalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TechLabs final project(problem 2 - group 8)

This Notebook is to work in the final project for Techlabs code-at-home course.
Members: Luca Wulfert, Laura Laks, Phillip Guske, Juliana Bedoya and Alejandro Echavarria

### libraries

In [0]:
!pip install -q ipympl
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import curdoc
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import Select
from bokeh.layouts import row
from bokeh.layouts import widgetbox
from bokeh.models import Slider
#!pip install -q holoviews==1.12.7
#import holoviews as hv
import ipywidgets as widgets
import warnings


%matplotlib inline
%matplotlib widget
warnings.filterwarnings('ignore')
#hv.extension('bokeh')

### Constants

In [0]:
confirmed_csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
death_csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
recovered_csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
countries_csv = 'https://raw.githubusercontent.com/rechavar/pyCovid_TechLabsProj/master/countries.csv'

### Usefull databases

In [0]:
df_confirmed = pd.read_csv(confirmed_csv_url)
df_death = pd.read_csv(death_csv_url)
df_recovered = pd.read_csv(recovered_csv_url)
df_countries = pd.read_csv(countries_csv)

In [0]:
df_confirmed.head(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7653,8145,8676,9216,9998,10582,11173,11831,12456,13036
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,949,964,969,981,989,998,1004,1029,1050,1076
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7377,7542,7728,7918,8113,8306,8503,8697,8857,8997
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,761,762,762,762,762,762,763,763,763,763
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,52,52,58,60,61,69,70,70,71,74
5,NaN,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,25,25,25,25,25,25,25,25,25,25
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,...,8809,9283,9931,10649,11353,12076,12628,13228,13933,14702
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,5041,5271,5606,5928,6302,6661,7113,7402,7774,8216
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,107,107,107,107,107,107,107,107,107,107
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,3081,3082,3084,3086,3087,3090,3092,3089,3090,3092


In [0]:
df_death.head(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,178,187,193,205,216,218,219,220,227,235
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,31,31,31,31,31,32,32,33,33,33
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,561,568,575,582,592,600,609,617,623,630
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,51,51,51,51,51,51,51,51,51,51
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,3,3,3,3,4,4,4,4,4,4
5,NaN,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,...,393,403,416,433,445,452,467,484,500,508
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,64,67,70,74,77,81,87,91,98,113
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,0,0,...,47,47,48,48,48,48,48,48,48,48


In [0]:
df_recovered.head(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,850,930,938,996,1040,1075,1097,1128,1138,1209
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,742,758,771,777,783,789,795,803,812,823
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,3746,3968,4062,4256,4426,4784,4747,4918,5129,5277
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,628,639,639,652,653,653,663,676,676,681
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,17,17,17,17,18,18,18,18,18,18
5,NaN,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,19,19,19,19,19,19,19,19,19,19
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,...,2872,2933,3032,3062,3530,3732,3999,4167,4349,4617
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,2164,2419,2581,2874,2936,3064,3145,3220,3255,3287
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,104,104,104,104,104,104,104,104,104,104
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,0,0,...,2631,2648,2648,2653,2653,2656,2661,2663,2670,2666


In [0]:
df_countries.head(10)

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
5,AL,41.153332,20.168331,Albania
6,AM,40.069099,45.038189,Armenia
7,AN,12.226079,-69.060087,Netherlands Antilles
8,AO,-11.202692,17.873887,Angola
9,AQ,-75.250973,-0.071389,Antarctica


In [0]:
all_dataframes = {'confirmed': df_confirmed,
                  'death': df_death,
                  'recovered': df_recovered,
                  'countries': df_countries}

### Data Cleaning

In [0]:
for key, df in all_dataframes.items():
  print('cleaning: {}'.format(key))

  if key == 'countries':
    continue

  df.drop(['Province/State', 'Lat', 'Long'], axis = 1, inplace = True)

  #Country renaming:
  df["Country/Region"].replace(["Taiwan*", "US", "Congo (Brazzaville)", "Congo (Kinshasa)"], 
                               ["Taiwan", "USA", "Republic of the Congo", "Democratic Republic of the Congo"],
                               inplace = True)
  #Set new index
  df.set_index('Country/Region', inplace = True)

  #Grouping together countries originally split into multiple Provinces/States
  #(Australia, Canada, China, Denmark, France, Netherlands, United Kingdom)
  df = df.groupby(by = df.index, axis = 0).sum()

  # Addition of a "global" row
  df.loc['Global'] = df.sum(axis=0)

  #other way
  df=df.T
  df.rename_axis(index='Date', columns="Country",inplace=True)
  df.index = pd.to_datetime(df.index)

  #Naming the Date index:
  df.index.rename("Date", inplace= True)

  #Creating a datetime index:
  df.index = pd.to_datetime(df.index)
  
  #Save new dataFrame
  all_dataframes[key] = df

# Change country names in df_countries
old_names = [ 'Myanmar [Burma]', 'Cape Verde',  "C?te d'Ivoire", 'Czech Republic',
             'Congo [DRC]',  'Swaziland', 'Vatican City',  'South Korea', 'Macedonia [FYROM]',
              'Congo [Republic]',  'S?o Tom? and Pr?ncipe',  'United States',  'Gaza Strip','Sudan']

new_names = ['Burma', 'Cabo Verde', "Cote d'Ivoire", 'Czechia', 'Democratic Republic of the Congo',
              'Eswatini',  'Holy See',  'Korea, South',  'North Macedonia',  'Republic of the Congo',
             'Sao Tome and Principe', 'USA',  'West Bank and Gaza','South Sudan']

all_dataframes['countries'] = all_dataframes['countries'].replace(old_names, new_names)

cleaning: confirmed
cleaning: death
cleaning: recovered
cleaning: countries


In [0]:
df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


#### Active cases

In [0]:
all_dataframes['active'] = all_dataframes['confirmed'] - all_dataframes['recovered'] - all_dataframes['death']
all_dataframes

{'confirmed': Country     Afghanistan  Albania  Algeria  Andorra  Angola  \
 Date                                                         
 2020-01-22            0        0        0        0       0   
 2020-01-23            0        0        0        0       0   
 2020-01-24            0        0        0        0       0   
 2020-01-25            0        0        0        0       0   
 2020-01-26            0        0        0        0       0   
 ...                 ...      ...      ...      ...     ...   
 2020-05-24        10582      998     8306      762      69   
 2020-05-25        11173     1004     8503      763      70   
 2020-05-26        11831     1029     8697      763      70   
 2020-05-27        12456     1050     8857      763      71   
 2020-05-28        13036     1076     8997      763      74   
 
 Country     Antigua and Barbuda  Argentina  Armenia  Australia  Austria  ...  \
 Date                                                                     ...   
 202

### Proyections

#### Getting the R7 coefficent

Calculating the R-value for any given day: We're using the method the Robert-Koch-Institute uses - It is done by dividing the sum of new cases from the last seven days by the sum of new cases four days before that. We will be calling this value R_7 Four days is the generational interval, which is the time it takes on average for one infection to be transmitted to the next generation (because one isn't infectious right away). The seven days is used to smoothen the curve by reducing noise, so that the R-value does not change so much from day to day.

In [0]:
def create_country(country):
  country_confirmed = all_dataframes['confirmed'].loc[:,[country]]
  country_confirmed.rename(columns={country : 'Total cases'}, inplace= True)
  country_confirmed.rename_axis(columns=country, inplace=True)

  country_confirmed["New cases"]=country_confirmed["Total cases"]-country_confirmed.shift(1)["Total cases"]
  country_confirmed["R_7"]=(country_confirmed["Total cases"]-country_confirmed.shift(7)["Total cases"])/(country_confirmed.shift(4)["Total cases"]-country_confirmed.shift(11)["Total cases"])

  return (country_confirmed)

In [0]:
country_dataframes = {country: create_country(country) for country in all_dataframes['confirmed'].columns}

Projection

#### Machine learning approches

In [0]:
print('puto')

puto


### Visualizing

#### Static data visualization

In [0]:
all_dataframes

{'confirmed': Country     Afghanistan  Albania  Algeria  Andorra  Angola  \
 Date                                                         
 2020-01-22            0        0        0        0       0   
 2020-01-23            0        0        0        0       0   
 2020-01-24            0        0        0        0       0   
 2020-01-25            0        0        0        0       0   
 2020-01-26            0        0        0        0       0   
 ...                 ...      ...      ...      ...     ...   
 2020-05-24        10582      998     8306      762      69   
 2020-05-25        11173     1004     8503      763      70   
 2020-05-26        11831     1029     8697      763      70   
 2020-05-27        12456     1050     8857      763      71   
 2020-05-28        13036     1076     8997      763      74   
 
 Country     Antigua and Barbuda  Argentina  Armenia  Australia  Austria  ...  \
 Date                                                                     ...   
 202

In [0]:
for i in all_dataframes['confirmed']:
  print('The country is: ', i)
  plt.figure(6)
  plt.plot(all_dataframes['confirmed'].index, all_dataframes['confirmed'][i], color = 'b', label = 'Confirmed cases')
  plt.plot(all_dataframes['death'].index, all_dataframes['death'][i], color = 'r', label = 'death cases')
  plt.plot(all_dataframes['active'].index, all_dataframes['active'][i], color = 'g', label = 'active cases')
  plt.plot(all_dataframes['recovered'].index, all_dataframes['recovered'][i], color = 'k', label = 'Confirmed cases')
  plt.legend(loc = 'best')
  plt.show()

The country is:  Afghanistan


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The country is:  Albania
The country is:  Algeria
The country is:  Andorra
The country is:  Angola
The country is:  Antigua and Barbuda
The country is:  Argentina
The country is:  Armenia
The country is:  Australia
The country is:  Austria
The country is:  Azerbaijan
The country is:  Bahamas
The country is:  Bahrain
The country is:  Bangladesh
The country is:  Barbados
The country is:  Belarus
The country is:  Belgium
The country is:  Belize
The country is:  Benin
The country is:  Bhutan
The country is:  Bolivia
The country is:  Bosnia and Herzegovina
The country is:  Botswana
The country is:  Brazil
The country is:  Brunei
The country is:  Bulgaria
The country is:  Burkina Faso
The country is:  Burma
The country is:  Burundi
The country is:  Cabo Verde
The country is:  Cambodia
The country is:  Cameroon
The country is:  Canada
The country is:  Central African Republic
The country is:  Chad
The country is:  Chile
The country is:  China
The country is:  Colombia
The country is:  Comoros

#### Interactive visualization

##### Wold map plot

In [0]:
#rename df_countries columns for merge
all_dataframes['countries'].rename(columns= {'country': 'shortcut', 'name': 'Country'},inplace = True)
all_dataframes['countries'].head()

,shortcut,latitude,longitude,Country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [0]:
all_dataframes['map'] = pd.DataFrame()
all_dataframes['map']['Active'] = all_dataframes['active'].iloc[-1,:]
all_dataframes['map']['Death'] = all_dataframes['death'].iloc[-1,:]
all_dataframes['map']['Confirmed'] = all_dataframes['confirmed'].iloc[-1,:]
all_dataframes['map'].head()

,Active,Death,Confirmed
Country,,,
Afghanistan,11592,235,13036
Albania,220,33,1076
Algeria,3090,630,8997
Andorra,31,51,763
Angola,52,4,74


In [0]:
all_dataframes['map'] = pd.merge(all_dataframes['map'],all_dataframes['countries'],on='Country')
all_dataframes['map']

,Country,Active,Death,Confirmed,shortcut,latitude,longitude
0,Afghanistan,11592,235,13036,AF,33.939110,67.709953
1,Albania,220,33,1076,AL,41.153332,20.168331
2,Algeria,3090,630,8997,DZ,28.033886,1.659626
3,Andorra,31,51,763,AD,42.546245,1.601554
4,Angola,52,4,74,AO,-11.202692,17.873887
...,...,...,...,...,...,...,...
180,West Bank and Gaza,75,3,446,GZ,31.354676,34.308825
181,Western Sahara,2,1,9,EH,24.215527,-12.885834
182,Yemen,210,57,278,YE,15.552727,48.516388
183,Zambia,271,7,1057,ZM,-13.133897,27.849332


In [0]:
#Circles sizes for each country
all_dataframes['map']['sizeOfCircles'] = all_dataframes['map']["Active"]/all_dataframes['map']["Active"].max()*500
all_dataframes['map'].loc[all_dataframes['map']['sizeOfCircles'] < 8, 'sizeOfCircles'] = 8

In [0]:
#setting up the world map in bokeh

#get the coordinates from https://carto.com/
tile_provider = get_provider(Vendors.CARTODBPOSITRON)

p = figure(title="Interactive corona world map",
           x_range=(-2000000, 6000000),
           y_range=(-1000000, 7000000),
           x_axis_type="mercator",
           y_axis_type="mercator")

p.add_tile(tile_provider)

#Transform LON/LAT coordinates in web mercator format'
#function to transform
def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):

      k = 6378137
      df["x"] = df[lon] * (k * np.pi/180.0)
      df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

      return df

#rename columns for the function
all_dataframes['map'].rename(columns={"longitude": "LON", "latitude": "LAT"}, inplace=True)

#use the function 
wgs84_to_web_mercator(all_dataframes['map'])

#make it interactive

all_dataframes['map'].drop(columns=['LAT', 'LON'])

#convert the dataframe into a ColumnDataSource so that bokeh can work with it
psource = ColumnDataSource(all_dataframes['map'])

p.circle('x', 'y', source=psource, size='sizeOfCircles',
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05,
         legend_label="Number of active cases")

#create hover tool
my_hover = HoverTool()
my_hover.tooltips = [('Name of the country', '@Country'),
                     ('Number of active cases', '@Active'),
                     ('Number of death', '@Death'),
                     ('Number of confirmed cases', '@Confirmed')]
p.add_tools(my_hover)


show(p)

In [0]:
all_dataframes['map']

,Country,Active,Death,Confirmed,shortcut,LAT,LON,sizeOfCircles,x,y
0,Afghanistan,11592,235,13036,AF,33.939110,67.709953,8.0,7.537437e+06,4.020629e+06
1,Albania,220,33,1076,AL,41.153332,20.168331,8.0,2.245128e+06,5.034984e+06
2,Algeria,3090,630,8997,DZ,28.033886,1.659626,8.0,1.847487e+05,3.253247e+06
3,Andorra,31,51,763,AD,42.546245,1.601554,8.0,1.782842e+05,5.243159e+06
4,Angola,52,4,74,AO,-11.202692,17.873887,8.0,1.989712e+06,-1.255101e+06
...,...,...,...,...,...,...,...,...,...,...
180,West Bank and Gaza,75,3,446,GZ,31.354676,34.308825,8.0,3.819241e+06,3.678897e+06
181,Western Sahara,2,1,9,EH,24.215527,-12.885834,8.0,-1.434444e+06,2.779693e+06
182,Yemen,210,57,278,YE,15.552727,48.516388,8.0,5.400820e+06,1.752983e+06
183,Zambia,271,7,1057,ZM,-13.133897,27.849332,8.0,3.100173e+06,-1.475034e+06


# World map with timeline

In [0]:
#initial creation of the df all_dataframes["timeline_map"]

#get the numbers for death confirmed and active
all_dataframes['timeline_map'] = pd.DataFrame()
all_dataframes['timeline_map']['Active'] = all_dataframes['active'].iloc[0]
all_dataframes['timeline_map']['Death'] = all_dataframes['death'].iloc[0]
all_dataframes['timeline_map']['Confirmed'] = all_dataframes['confirmed'].iloc[0]

#merge of the dfs
all_dataframes['timeline_map'] = pd.merge(all_dataframes['timeline_map'],all_dataframes['countries'],on='Country')
all_dataframes['timeline_map']

#compute size of circles
all_dataframes['timeline_map']['sizeOfCircles'] = all_dataframes['timeline_map']["Active"]/all_dataframes['timeline_map']["Active"].max()*500
all_dataframes['timeline_map'].loc[all_dataframes['timeline_map']['sizeOfCircles'] < 8, 'sizeOfCircles'] = 8

all_dataframes['timeline_map'].head()

,Country,Active,Death,Confirmed,shortcut,latitude,longitude,sizeOfCircles
0,Afghanistan,0,0,0,AF,33.939110,67.709953,8.0
1,Albania,0,0,0,AL,41.153332,20.168331,8.0
2,Algeria,0,0,0,DZ,28.033886,1.659626,8.0
3,Andorra,0,0,0,AD,42.546245,1.601554,8.0
4,Angola,0,0,0,AO,-11.202692,17.873887,8.0


In [0]:
from bokeh.io import curdoc, show
from bokeh.models import ColumnDataSource, Grid, Line, LinearAxis, Plot

#setting up the world map in bokeh

#get the coordinates from https://carto.com/
tile_provider = get_provider(Vendors.CARTODBPOSITRON)

p = figure(title="Timeline of corona spreading",
           x_range=(-2000000, 6000000),
           y_range=(-1000000, 7000000),
           x_axis_type="mercator",
           y_axis_type="mercator")

p.add_tile(tile_provider)

#Transform LON/LAT coordinates in web mercator format'
#function to transform
def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):

      k = 6378137
      df["x"] = df[lon] * (k * np.pi/180.0)
      df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

      return df

#rename columns for the function
all_dataframes['timeline_map'].rename(columns={"longitude": "LON", "latitude": "LAT"}, inplace=True)

#use the function 
wgs84_to_web_mercator(all_dataframes['timeline_map'])

#make it interactive

all_dataframes['timeline_map'].drop(columns=['LAT', 'LON'])

#convert the dataframe into a ColumnDataSource so that bokeh can work with it
psource = ColumnDataSource(all_dataframes['timeline_map'])

p.circle('x', 'y', source=psource, size='sizeOfCircles',
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05,
         legend_label="Number of active cases")

#create hover tool
my_hover = HoverTool()
my_hover.tooltips = [('Name of the country', '@Country'),
                     ('Number of active cases', '@Active'),
                     ('Number of death', '@Death'),
                     ('Number of confirmed cases', '@Confirmed')]

p.add_tools(my_hover)


#define slider
slider1 = Slider(title='Days since the outbreak (1-22-2020)',
                 start=1,
                 end=len(all_dataframes['confirmed']),
                 step=1,
                 value=1)

#**********************************SOMEWHERE IN THIS FUNCTION IS THE MISTAKE***************************************
#**********************************The slider isn't changing the size of the circles!******************************
#HELPFUL LINKS
#LINK ONE: https://vishalmnemonic.github.io/_posts/2018-11-09-DC13/
#LINK TWO: https://www.programcreek.com/python/example/106840/bokeh.models.Slider

#create function which updates the columns in all_dataframes['timeline_map'] based on the slider value
def callback(attr, old, new):
    # Read the current value of the slider: ScaleDate
    ScaleDay = slider1.value
    # get the date related number of each df
    all_dataframes['timeline_map']['Active'] = all_dataframes['active'].iloc[len(all_dataframes['confirmed'])-ScaleDay,:]
    all_dataframes['timeline_map']['Death'] = all_dataframes['death'].iloc[len(all_dataframes['confirmed'])-ScaleDay,:]
    all_dataframes['timeline_map']['Confirmed'] = all_dataframes['confirmed'].iloc[len(all_dataframes['confirmed'])-ScaleDay,:]
    # Update the Sizeofcircles column in all_dataframes['timeline_map']
    all_dataframes['timeline_map']['sizeOfCircles'] = all_dataframes['timeline_map']["Active"]/all_dataframes['timeline_map']["Active"].max()*500
    all_dataframes['timeline_map'].loc[all_dataframes['timeline_map']['sizeOfCircles'] < 8, 'sizeOfCircles'] = 8
    # Update datasource with the new data values
    psource.data = ColumnDataSource(all_dataframes['timeline_map'])

#Attach the callback to the 'value' property of slider
slider.on_change('value',callback)

#****************************************************************************************************************

# Create layout and show 
layout = column(p,widgetbox(slider1))
curdoc().add_root(layout)
show(layout)

#### Interactive plots

In [0]:
# line plot
line_plot = figure(plot_width=500, plot_height=325, title='Covid-19 Cases in Colombia', 
                   x_axis_label='Date', y_axis_label='Active Cases',x_axis_type='datetime', y_axis_type = "linear")

line_plot.line('Date', 'Colombia',source = ColumnDataSource(all_dataframes['active']), color='red', legend='Active Cases', line_width=2)
line_plot.line('Date', 'Colombia',source = ColumnDataSource(all_dataframes['death']), color='green', legend='Death Cases', line_width=2)
line_plot.line('Date', 'Colombia',source = ColumnDataSource(all_dataframes['recovered']), color='yellow', legend='recovered Cases', line_width=2)

my_hover = HoverTool()
my_hover.tooltips = [('the Date is','@Date{%F}'),('the number of Active cases','@{}'.format('Colombia'))]
my_hover.formatters = {'Date': 'datetime'}
line_plot.add_tools(my_hover)

show(line_plot)

In [0]:
def plot_country(country):
  line_plot = figure(plot_width=500, plot_height=325, title='Covid-19 Cases in {}'.format(country), 
                   x_axis_label='Date', y_axis_label='Active Cases',x_axis_type='datetime', y_axis_type = "linear")
  
  line_plot.line('Date', '{}'.format(country),source = ColumnDataSource(all_dataframes['active']), color='red', legend='Active Cases', line_width=2)
  line_plot.line('Date', '{}'.format(country),source = ColumnDataSource(all_dataframes['death']), color='green', legend='Death Cases', line_width=2)
  line_plot.line('Date', '{}'.format(country),source = ColumnDataSource(all_dataframes['recovered']), color='yellow', legend='recovered Cases', line_width=2)

  my_hover = HoverTool()
  my_hover.tooltips = [('the Date is','@Date{%F}'),('the number of Active cases','@{}'.format(country))]
  my_hover.formatters = {'Date': 'datetime'}
  line_plot.add_tools(my_hover)
  return line_plot